## Summary
learn a reduced model for when you apply the POD seperately.

In [2]:
import numpy as np
import sys
sys.path.append('./../')
import torch
import torch.nn as nn
import torch.optim as optim
import torch
from torch.utils.data import TensorDataset, DataLoader
from sklearn.model_selection import train_test_split
from ProcessFunctions import find_Aisv2,Find_a_i,find_Aisv2_onlyv
from torch.utils.data import TensorDataset, DataLoader
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import torch.nn.functional as F
from scipy.interpolate import interp1d
import seaborn as sns 
import cte_eq

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cpu


In [ ]:
T_final=10500
Ntout=cte_eq.Ntout
Nxout=cte_eq.Nxout
drs=0.012
rho=13 # by looking at the plot in the FindDissipationBall.ipynb
data_dir='/central/groups/astuart/hkaveh/Data/LearnROM/transfer/MainSimulation2D_Tf'+str(T_final)+"Nt="+str(Ntout)+"Nx="+str(Nxout)+'PODonlyonV'+'drs'+str(drs)+".npz"
PODmodes=np.load(data_dir)
U_v=PODmodes['U']
S_v=PODmodes['S']
# VT_v=PODmodes['VT']
q_bar_v=PODmodes['q_bar']

In [ ]:
# Loading POD modes


data_dir='/central/groups/astuart/hkaveh/Data/LearnROM/transfer/MainSimulation2D_Tf'+str(T_final)+"Nt="+str(Ntout)+"Nx="+str(Nxout)+'PODonlyontheta'+'drs'+str(drs)+".npz"

PODmodes=np.load(data_dir)
U_theta=PODmodes['U']
S_theta=PODmodes['S']
# VT_theta=PODmodes['VT']
q_bar_theta=PODmodes['q_bar']

In [ ]:
# Loading data from "/central/groups/astuart/hkaveh/Data/LearnROM/transfer/SampleSimulation_Tf_2D'+str(T_final_run)+"Nt="+str(cte_eq.Ntout)+"N_m"+str(N_m)+"coeff"+str(coeff)+"number"+str(index)"
T_final_run = 350 # each initial condition is simulated for 250 years
Nt=5     # it is recorded every Nt time stes
N_m_v=20 # number of modes that they have considered
N_m_theta=20 # number of modes that they have considered
coeff=1 # How the inital conditions are from the chaotic attractor
number=10 # Which one do you want to load
N_cut=2000
# loading time series:
N_m_load=30

X_full=np.empty((1,N_m_load*2))
Y_full=np.empty((1,N_m_load*2))


for number in range(1,101):
    data_dir='/central/groups/astuart/hkaveh/Data/LearnROM/transfer/SampleSimulation_Tf_2D'+str(T_final_run)+"Nt="+str(cte_eq.Ntout)+"N_m"+str(N_m_load)+"coeff"+str(coeff)+"number"+str(number)+".npz"
    data_smaple=np.load(data_dir)

    V_ox=data_smaple['array1']
    theta_ox=data_smaple['array2']
    t_ox=data_smaple['array3']
    # removing data with probability p
    xi=0
    p_remove=np.tanh(xi*np.max(V_ox,axis=1))
    Nt_sample=p_remove.shape[0]
    print( "number of points in this member is:", Nt_sample)
    keep_mask_sample=np.random.rand(Nt_sample) > p_remove
    # We work with log10 of V_ox and theta_ox:
    V_ox=np.log10(V_ox)
    theta_ox=np.log10(theta_ox)
    A_v,P_v=find_Aisv2_onlyv(U_v,V_ox,q_bar_v,N_m_load) # you dont need P in general, but I need it to check if I do everything correctly
    A_theta,P_theta=find_Aisv2_onlyv(U_theta,theta_ox,q_bar_theta,N_m_load)
    # concatenating the two arrays
    A=np.concatenate((A_v,A_theta),axis=1)
    X_onesimulation=A[:-1,:]         # current step
    Y_onesimulation=A[1:,:]-A[:-1,:] # next step minus the current step
    print("shape of X_onesimulation before is",X_onesimulation.shape)
    X_onesimulation=X_onesimulation[keep_mask_sample[:-1]]
    Y_onesimulation=Y_onesimulation[keep_mask_sample[:-1]]
    print("shape of X_onesimulation after is",X_onesimulation.shape)

    X_full=np.append(X_full,X_onesimulation,axis=0)
    Y_full=np.append(Y_full,Y_onesimulation,axis=0)
    print("number is " +str(number)+ " size of data in this simulation is " +str(X_onesimulation.shape[0])+" total size is " + str(X_full.shape[0]))

# removing the first row which is empty
X_full=X_full[1:,:]
Y_full=Y_full[1:,:]

# print(np.max(X_full))
print(np.max(Y_full))
# Loading the the numpy arrays Input and Output which are generated for imposing dissipiation far away from the attractor
# # commenting here:
# data_dir="/central/groups/astuart/hkaveh/Data/LearnROM/EarthquakeEllipticShell_radi_outer"+str(radi_outer)+"radi_inner"+str(radi_inner)+"PODappliedseperately.npz"

# data_smaple=np.load(data_dir)
# Input_X=data_smaple['Input']
# Output_Y=data_smaple['Output']
# # appending the Input_X to the X_full and Output_Y to the Y_full
# # appending the Input_X to the X_full and Output_Y to the Y_full, but append from the begining of the array
# print(np.max(Output_Y))
# X_full=np.append(Input_X,X_full,axis=0)
# Y_full=np.append(Output_Y,Y_full,axis=0)
# # untill here

# deleting some colomns from X_full and Y_full that are should not be included based on N_m_v and N_m_theta
# index of the columns that should be deleted
index_delete=np.array([])
if N_m_v<N_m_load:
    index_delete=np.append(index_delete,np.arange(N_m_v,N_m_load))
if N_m_theta<N_m_load:
    index_delete=np.append(index_delete,np.arange(N_m_load+N_m_theta,2*N_m_load))


# Convert the array elements to integers
index_delete = index_delete.astype(int)

X_full=np.delete(X_full,index_delete,axis=1)
Y_full=np.delete(Y_full,index_delete,axis=1)
print("index of the columns that are deleted are:")
print(index_delete)

print(X_full.shape)
print("size of the data is " +str(X_full.shape[0]))
# 50, 55, 59, 63, 72, 73,79,81

In [ ]:
# loading Sigma_v and Sigma_theta and then divide the X_onesimulation by them and then find the norm and then plot it.
# The directory is : directory='/central/groups/astuart/hkaveh/Data/LearnROM/transfer/'+'sigma'+str(T_final)+"Nt="+str(Ntout)+"Nx="+str(Nxout)+'PODonlyonV'+'drs'+str(drs)+".npz"
# and they were saved using np.savez(directory,Sigma_v=Sigma_v,Sigma_theta=Sigma_theta,Nt2=Nt2)
# Lets load them:
N_plot=int(6e4)
directory='/central/groups/astuart/hkaveh/Data/LearnROM/transfer/'+'sigma'+str(T_final)+"Nt="+str(Ntout)+"Nx="+str(Nxout)+'PODonlyonV'+'drs'+str(drs)+".npz"
Sigma_v=np.load(directory)['Sigma_v']
Sigma_theta=np.load(directory)['Sigma_theta']
Sigma=np.concatenate((Sigma_v,Sigma_theta))
X_full_scaled=X_full[:N_plot,:]/Sigma
norm_X_full_scaled=np.linalg.norm(X_full_scaled,axis=1)
# norm_Y_full_scaled=np.linalg.norm(Y_full_scaled,axis=1)
plt.plot(norm_X_full_scaled)
plt.axhline(y=rho, color='r', linestyle='-')
# plt.plot(norm_Y_full_scaled)

# plt.xlim(left=0,right=4)

In [ ]:
from scipy.stats import shapiro

for i in range(10):
    stat, p = shapiro(X_onesimulation[:, i])
    print(f"PC {i+1}: Shapiro p-value = {p}")

In [ ]:
from scipy.stats import shapiro

for i in range(10):
    stat, p = shapiro(X_onesimulation[:, i])
    print(f"PC {i+1}: Shapiro p-value = {p}")

In [ ]:
plt.plot(X_onesimulation[:, 0])


In [ ]:
plt.hist(Y_onesimulation[:,3])
plt.yscale("log")

In [ ]:
plt.plot(Y_onesimulation[:,0])
# find v from alpha
log_v=np.zeros((X_onesimulation.shape[0],1024))
for i in range(X_onesimulation.shape[0]):
    log_v[i,:]=U_v[:,:30]@X_onesimulation[i,:30]+q_bar_v.reshape(1024,)
max_log_v=np.max(log_v,axis=1)
plt.plot(max_log_v)
# plt.xlim(right=1000)

In [ ]:
# Plot the histogram of the  norm of X_full along each row
norm_X_full=np.linalg.norm(X_full,axis=1)
plt.hist(norm_X_full,bins=100)
plt.title("Histogram of the norm of X_full along each row")

In [ ]:
# Create a figure and axis objects
fig, axs = plt.subplots(3, 5, figsize=(15, 10))

# Flatten the axs array so that we can iterate over it easily
axs = axs.flatten()

# Plot the KDE plot for each column of X_full
for i in range(15):
    sns.kdeplot(X_full[:, i], ax=axs[i])
    axs[i].set_title(r'$\alpha_{{{}}}$'.format(i))

# Adjust layout and show plot
plt.tight_layout()
plt.show()

# Preprocessing Data

In [ ]:
X_cut=X_full[-N_cut:,:]
Y_cut=Y_full[-N_cut:,:]
X=X_full[1:-N_cut,:] # Also, removing the first element
Y=Y_full[1:-N_cut,:] 
# rescale
# # comment from here:
# max_X=np.max(X)
# max_y=np.max(Y)

# X=X/max_X
# Y=Y/max_y
# X_cut=X_cut/max_X
# Y_cut=Y_cut/max_y
# # untill here

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)
# Convert data to PyTorch tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
Y_train_tensor = torch.tensor(Y_train, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
Y_test_tensor = torch.tensor(Y_test, dtype=torch.float32)
# Create datasets and dataloaders
train_dataset = TensorDataset(X_train_tensor, Y_train_tensor)
test_dataset = TensorDataset(X_test_tensor, Y_test_tensor)
batch_size = 64
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [ ]:
print("Target Y range:")
print("  Min:", Y_train_tensor.min().item())
print("  Max:", Y_train_tensor.max().item())
print("  Std:", Y_train_tensor.std().item())

# Defining Model

In [ ]:
# %% Defining model
# class Forwardmap(nn.Module):
#     def __init__(self,N_m,dropout_rate=0.2):
#         super().__init__()
#         self.hidden1=nn.Linear(N_m,2*N_m)
#         self.act1=nn.Tanh()
#         self.hidden2=nn.Linear(2*N_m,4*N_m)
#         self.act2=nn.Tanh()
#         self.hidden3=nn.Linear(4*N_m,4*N_m)
#         self.act3=nn.Tanh()
#         self.hidden4=nn.Linear(4*N_m,2*N_m)
#         self.act4=nn.Tanh()
#         self.output=nn.Linear(2*N_m,N_m)
#         self.dropout = nn.Dropout(p=dropout_rate)
#     def forward(self,x):
#         x=self.act1(self.hidden1(x))
#         x = self.dropout(x)
#         x=self.act2(self.hidden2(x))
#         x = self.dropout(x)
#         x=self.act3(self.hidden3(x))
#         x = self.dropout(x)
#         x=self.act4(self.hidden4(x))
#         x = self.dropout(x)
#         x=(self.output(x))
#         return x


class Forwardmap(nn.Module):
    def __init__(self, N_m, dropout_rate=0.2, negative_slope=0.01):
        super().__init__()
        self.hidden1 = nn.Linear(N_m, 2 * N_m)
        self.act1 = nn.LeakyReLU(negative_slope)
        self.hidden2 = nn.Linear(2 * N_m, 4 * N_m)
        self.act2 = nn.LeakyReLU(negative_slope)
        self.hidden3 = nn.Linear(4 * N_m, 4 * N_m)
        self.act3 = nn.LeakyReLU(negative_slope)
        self.hidden4 = nn.Linear(4 * N_m, 2 * N_m)
        self.act4 = nn.LeakyReLU(negative_slope)
        self.output = nn.Linear(2 * N_m, N_m)
        self.dropout = nn.Dropout(p=dropout_rate)

    def forward(self, x):
        x = self.act1(self.hidden1(x))
        x = self.dropout(x)
        x = self.act2(self.hidden2(x))
        x = self.dropout(x)
        x = self.act3(self.hidden3(x))
        x = self.dropout(x)
        x = self.act4(self.hidden4(x))
        x = self.dropout(x)
        x = self.output(x)
        return x

model=Forwardmap(N_m_v+N_m_theta)
print(model)
# Move the model to GPU
model = model.to(device)
# loss_fn = torch.nn.SmoothL1Loss()

loss_fn=nn.MSELoss()
optimizer=optim.Adam(model.parameters(),lr=0.001)
# Define the scheduler for changing learning rate every 20 epochs
lr_decay_factor = 0.5
lr_scheduler_step = 50
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=lr_scheduler_step, gamma=lr_decay_factor)

# Optimize, save and evaluate the ML model

In [ ]:
num_epochs=249
for epoch in range(num_epochs):
    model.train()
    for inputs,targets in train_dataloader:
        # Move inputs and targets to GPU
        inputs, targets = inputs.to(device), targets.to(device)
        optimizer.zero_grad()
        outputs=model(inputs)
        loss=loss_fn(outputs,targets)
        loss.backward()
        optimizer.step()
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.8f}')
    scheduler.step()
    current_lr = optimizer.param_groups[0]['lr']
    print(current_lr)

In [ ]:
# Evaluate the model on the test set
model.eval()
total_loss = 0.0
with torch.no_grad():
    for inputs, targets in test_dataloader:
        inputs, targets = inputs.to(device), targets.to(device)
        outputs = model(inputs)
        loss = loss_fn(outputs, targets)
        total_loss += loss.item() * inputs.size(0)

mean_loss = total_loss / len(test_dataset)
print(f'Test Loss: {mean_loss:.4f}')


In [ ]:
# save the model in the group directory
version=0
torch.save(model.state_dict(), "/central/groups/astuart/hkaveh/Data/LearnROM/Earthquake2D_ROM_POD_separate_onV_theta"+str(version)+"N_m_v"+str(N_m_v)+"N_m_theta"+str(N_m_theta)+".pt")

In [ ]:
X_cut_tensor = torch.tensor(X_cut, dtype=torch.float32)
device = torch.device( "cpu")
print(device)
model.to(device)
index=np.linspace(1,N_cut-1,N_cut-1)
Pred=model(X_cut_tensor)
plt.plot(Y_cut[:,0],label='True')
plt.plot(Pred.detach().numpy()[:,0],label='Predicted')
plt.xlabel('iteration')
plt.ylabel(r'$\alpha_0(t_n+\Delta t_n)-\alpha_0(t_n)$')
plt.xlim(left=0,right=2000)
plt.legend()

In [ ]:
X_cut_tensor = torch.tensor(X_cut, dtype=torch.float32)
device = torch.device( "cpu")
print(device)
model.to(device)
index=np.linspace(1,N_cut-1,N_cut-1)
Pred=model(X_cut_tensor)
plt.plot(Y_cut[:,0],label='True')
plt.plot(Pred.detach().numpy()[:,0],label='Predicted')
plt.xlabel('iteration')
plt.ylabel(r'$\alpha_0(t_n+\Delta t_n)-\alpha_0(t_n)$')
plt.xlim(left=0,right=500)
plt.legend()

In [ ]:
X_cut_tensor = torch.tensor(X_cut, dtype=torch.float32)
device = torch.device( "cpu")
print(device)
model.to(device)
index=np.linspace(1,N_cut-1,N_cut-1)
Pred=model(X_cut_tensor)
plt.plot(Y_cut[:,0],label='True')
plt.plot(Pred.detach().numpy()[:,0],label='Predicted')
plt.xlabel('iteration')
plt.ylabel(r'$\alpha_0(t_n+\Delta t_n)-\alpha_0(t_n)$')
plt.xlim(left=0,right=250)
plt.legend()

In [ ]:
X_cut_tensor = torch.tensor(X_cut, dtype=torch.float32)
device = torch.device( "cpu")
print(device)
model.to(device)
index=np.linspace(1,N_cut-1,N_cut-1)
Pred=model(X_cut_tensor)
plt.plot(Y_cut[:,0],label='True')
plt.plot(Pred.detach().numpy()[:,0],label='Predicted')
plt.xlabel('iteration')
plt.ylabel(r'$\alpha_0(t_n+\Delta t_n)-\alpha_0(t_n)$')
plt.xlim(left=0,right=250)
plt.legend()

In [ ]:
X_cut_tensor = torch.tensor(X_cut, dtype=torch.float32)
device = torch.device( "cpu")
print(device)
model.to(device)
index=np.linspace(1,N_cut-1,N_cut-1)
Pred=model(X_cut_tensor)
plt.plot(Y_cut[:,0],label='True')
plt.plot(Pred.detach().numpy()[:,0],label='Predicted')
plt.xlabel('iteration')
plt.ylabel(r'$\alpha_0(t_n+\Delta t_n)-\alpha_0(t_n)$')
plt.xlim(left=0,right=250)
plt.legend()

In [ ]:
Beta=5
lambda_dissipation=0.9
rho=10
x_plot_test=np.linspace(0,2*rho,100)
plt.plot(x_plot_test,1/(1+np.exp(Beta*(x_plot_test-rho))))
plt.axvline(x=rho, color='r')
plt.axvline(x=rho-1, color='r', linestyle='--')

In [ ]:
def dissipation_factor(xk,rho,Sigma,Beta=10):
    u=xk/Sigma # normalize by the std
    norm_u=torch.norm(u)
    output=1/(1+torch.exp(Beta*(norm_u-rho)))
    return output

In [ ]:
Sigma=np.concatenate((Sigma_v,Sigma_theta))
Sigma = torch.tensor(Sigma, dtype=torch.float32, device=device)

In [ ]:
xk=X_cut[0,:] # Picking one initial condition, this can be any point inside the chaotic attractor
N_m=N_m_v+N_m_theta
xk=torch.tensor(xk, dtype=torch.float32)
forecast=xk.detach().numpy().reshape(N_m,1)
for index in range(40*N_cut):
    # ykp1=(model(xk))*(max_y/max_X)
    dissipate=dissipation_factor(xk,rho,Sigma)
    ykp1=(model(xk))
    xkp1=(ykp1+xk)* dissipate + (1-dissipate)*(lambda_dissipation*xk) # equation 8 and 9 of the Learning Dissipative Dynamics in Chaotic Systems
    numpy_vector = xkp1.detach().numpy().reshape(N_m,1)
    forecast = np.hstack([forecast, numpy_vector])
    xk=xkp1


In [ ]:
plt.plot(forecast[0,:])
plt.plot(X_full[:,0])
plt.xlim(left=0,right=80000)
# plt.ylim(bottom=-5000,top=5000)

In [ ]:
plt.plot(forecast[1,:])
plt.plot(X_full[:,1])

plt.xlim(left=0,right=80000)

In [ ]:
plt.plot(forecast[4,:])
plt.plot(X_full[:,4])
plt.xlim(left=0,right=80000)

In [ ]:
# # I want to plot two figures here. 
# # figure 1
# # plot the time sereis of the norm of forecast
# norm_forecast=np.linalg.norm(forecast,axis=0)
# # I want to also grid the plot very fine
# plt.figure(figsize=(10,5))
# # set xlim
# plt.xlim([0,1500])
# plt.ylim([0,200])

# # plt.plot(norm_forecast*max_X)
# plt.plot(norm_forecast)

# plt.title("Norm of the forecast")
# plt.xlabel("iteration")
# plt.ylabel("Norm of the forecast")
# plt.grid(True)
# plt.show()
# alpha=forecast*max_X
# alpha_v=alpha[:N_m_v,:]
# alpha_theta=alpha[N_m_v:,:]
# # Loading POD modes for V and Theta
# N_m_v_load=30
# N_m_theta_load=30
# data_dir="/central/groups/astuart/hkaveh/Data/LearnROM/MainSimulation_Tf600Nt=5PODonlyonV.npz"
# PODmodes=np.load(data_dir)
# U_v=PODmodes['U']
# S_v=PODmodes['S']
# VT_v=PODmodes['VT']
# q_bar_v=PODmodes['q_bar']
# Sigma_v=np.diagonal(S_v)
# Nt2=VT_v.shape[0]
# Lambda_v=Sigma_v**2/Nt2
# Lambda_v=np.atleast_2d(Lambda_v[:N_m_v_load])
# Sigma_v=(np.sqrt(Lambda_v))


# data_dir="/central/groups/astuart/hkaveh/Data/LearnROM/MainSimulation_Tf600Nt=5PODonlyontheta.npz"
# PODmodes=np.load(data_dir)
# U_theta=PODmodes['U']
# S_theta=PODmodes['S']
# VT_theta=PODmodes['VT']
# q_bar_theta=PODmodes['q_bar']
# Sigma_theta=np.diagonal(S_theta)
# Nt2=VT_theta.shape[0]
# Lambda_theta=Sigma_theta**2/Nt2
# Lambda_theta=np.atleast_2d(Lambda_theta[:N_m_theta_load])
# Sigma_theta=(np.sqrt(Lambda_theta))
# Sigma_v=Sigma_v[0,:N_m_v]
# Sigma_theta=Sigma_theta[0,:N_m_theta]
# normalized_alpha_v=alpha_v/Sigma_v[:, np.newaxis]
# normalized_alpha_theta=alpha_theta/Sigma_theta[:, np.newaxis]
 
 
# apppend the normalized_alpha_v and normalized_alpha_theta
# normalized_alpha=np.append(normalized_alpha_v,normalized_alpha_theta,axis=0)
# normalized_alpha.shape
# # plot the timeseries of the norm of the normalized_alpha
# norm_normalized_alpha=np.linalg.norm(normalized_alpha,axis=0)
# plt.figure(figsize=(10,5))
# plt.plot(norm_normalized_alpha)
# plt.title("Norm of the normalized_alpha")
# plt.xlabel("iteration")
# plt.xlim([0,1500])
# plt.ylim([0,100])

# plt.ylabel("Norm of the normalized_alpha")
# plt.grid(True)
# plt.show()